## **Step 1) Do necessary preprocessing of data**

In [80]:
import pandas as pd
pd.set_option("display.max.columns", None)#show all columns of dataframe when displayed
df=pd.read_csv("/content/cumulative.csv")#read the csv file and convert it to dataframe
df=df.drop(["rowid"],axis=1)
#convert the file contents to dataframe and drop unnecessary columns

In [81]:
#since koi_pdisposition is related to the koi_disposition column it is best to allow it to be read by classifiers so convert it into numbers
df_one = pd.get_dummies(df["koi_pdisposition"])#converts the three possibilities of koi_pdisposition into three columns with value being 0 or 1 for each column and 1 being true
df_two = pd.concat((df_one, df), axis=1)#combine the columns in the df_one created above with the original df
df_two = df_two.drop(["koi_pdisposition"], axis=1)#drop column koi_pdisposition with string values
df_two = df_two.drop(["FALSE POSITIVE"], axis=1)#drop one column so only one column can be koi_pdisposition
df = df_two.rename(columns={"CANDIDATE": "koi_pdisposition"})#rename column to original column name with CANDIDATE=1 and FALSE POSITIVE=0
#this only works since there are only 2 actual values of the 3 possibilities of koi_pdisposition in the dataframe, if there were 3 the code would differ
column_to_move = df.pop('koi_pdisposition')
df.insert(5, 'koi_pdisposition', column_to_move)
#before the 2 lines above, the column was the first column so place it in its original position for display purposes

In [82]:
display(df.head(10))

,kepid,kepoi_name,kepler_name,koi_disposition,koi_score,koi_pdisposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,1.000,1,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,0.969,1,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,FALSE POSITIVE,0.000,0,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,0.000,0,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,1.000,1,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
5,10872983,K00756.01,Kepler-228 d,CONFIRMED,1.000,1,0,0,0,0,11.094321,2.036000e-05,-2.036000e-05,171.201160,0.001410,-0.001410,0.538,0.030,-0.428,4.59450,0.06100,-0.06100,1517.5,24.2,-24.2,3.90,1.27,-0.42,835.0,NaN,NaN,114.81,112.85,-36.70,66.5,1.0,q1_q17_dr25_tce,6046.0,189.0,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,10872983,K00756.02,Kepler-228 c,CONFIRMED,1.000,1,0,0,0,0,4.134435,1.046000e-05,-1.046000e-05,172.979370,0.001900,-0.001900,0.762,0.139,-0.532,3.14020,0.06730,-0.06730,686.0,18.7,-18.7,2.77,0.90,-0.30,1160.0,NaN,NaN,427.65,420.33,-136.70,40.2,2.0,q1_q17_dr25_tce,6046.0,189.0,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
7,10872983,K00756.03,Kepler-228 b,CONFIRMED,0.992,1,0,0,0,0,2.566589,1.781000e-05,-1.781000e-05,179.554370,0.004610,-0.004610,0.755,0.212,-0.523,2.42900,0.16500,-0.16500,226.5,16.8,-16.8,1.59,0.52,-0.17,1360.0,NaN,NaN,807.74,793.91,-258.20,15.0,3.0,q1_q17_dr25_tce,6046.0,189.0,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
8,6721123,K00114.01,NaN,FALSE POSITIVE,0.000,0,0,1,1,0,7.361790,2.128000e-05,-2.128000e-05,132.250530,0.002530,-0.002530,1.169,7.133,-0.044,5.02200,0.13600,-0.13600,233.7,5.8,-5.8,39.21,6.45,-9.67,1342.0,NaN,NaN,767.22,349.28,-365.49,47.7,1.0,q1_q17_dr25_tce,6227.0,111.0,-124.0,3.986,0.182,-0.098,1.958,0.322,-0.483,298.86435,42.151569,12.660
9,10910878,K00757.01,Kepler-229 c,CONFIRMED,1.000,1,0,0,0,0,16.068647,1.088000e-05,-1.088000e-05,173.621937,0.000517,-0.000517,0.052,0.262,-0.052,3.53470,0.02410,-0.02410,4914.3,33.3,-33.3,5.76,0.22,-0.49,600.0,NaN,NaN,30.75,4.4

In [83]:
from sklearn.impute import SimpleImputer 
#drop unnecessary columns(string values) and target column to get trained upon features
features_all=df.drop(['koi_disposition','kepoi_name','kepler_name','koi_tce_delivname','koi_teq_err1','koi_teq_err2'],axis = 1)
#convert NaN values to 0 (this was the most successful strategy for me)
imputer = SimpleImputer(strategy='constant',fill_value=0)
features_all_imputed = imputer.fit_transform(features_all)

# **Step 2) Train a set of classifiers to predict target variable**

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#set target variable
target=df['koi_disposition']
#split dataset into training and testing
#test size is randomly selected 10 percent of rows since shuffle is set to true by default
#features to be used for prediction for first parameter and target to be predicted for 2nd
features_train, features_test, target_train, target_test = train_test_split(features_all_imputed,target, test_size=0.10)

In [85]:
#Random Forest Classifier with 75 estimators
forest = RandomForestClassifier(criterion='entropy',n_estimators=75,max_depth=None)
forest.fit(features_train,target_train)
pred = forest.predict(features_test)
print('Accuracy of random forest model: %.3f' % accuracy_score(target_test, pred))

Accuracy of random forest model: 0.917


In [87]:
#Decision Tree Classifier
tree = DecisionTreeClassifier(criterion='entropy')
tree.fit(features_train,target_train)
pred = tree.predict(features_test)
print('Accuracy of decision tree model: %.3f' % accuracy_score(target_test, pred))

Accuracy of decision tree model: 0.868


In [88]:
#KNN Classifier
knn = KNeighborsClassifier(n_neighbors=150)
knn.fit(features_train,target_train)
pred = knn.predict(features_test)
print('Accuracy of KNN model: %.3f' % accuracy_score(target_test, pred))

Accuracy of KNN model: 0.515


# **Step 3) Develop and Train an Ensemble model based on bagging**

In [89]:
import numpy as np
#assuming u ran train_test_split function above
#convert target train to array since bagging will involve selecting random rows so indexes must be used and current datatype of target_train is object
target_train = np.array(target_train)

In [90]:
subsets = []
#create 10 subsets of the features_train array
for i in range(10):
#select 2000 random rows from range of size of features_train and rows selected can be repeated
    chosen = np.random.choice(len(features_train), 2000, replace=True)
    features_subset = features_train[chosen]
    target_subset = target_train[chosen]
#add subsets of features array and target array to a 2D array where each features subsets indexes match target subsets indexes and array size is 10
    subsets.append((features_subset, target_subset))

In [91]:
#create an array for each classifier and add all 10 classifiers to their own array
dts = []
rfs=[]
knns=[]
#train each classifier on each subset of data
for i in range(len(subsets)):
    dt = DecisionTreeClassifier(criterion='entropy')
    dt.fit(subsets[i][0], subsets[i][1])#subsets[i][0] is a subset of features_train and subsets[i][1] is its corresponding subset of target_train
    dts.append(dt)
for i in range(len(subsets)):
    rf = RandomForestClassifier(criterion='entropy',n_estimators=75,max_depth=None)
    rf.fit(subsets[i][0], subsets[i][1])
    rfs.append(rf)
for i in range(len(subsets)):
    knn = KNeighborsClassifier(n_neighbors=50)
    knn.fit(subsets[i][0], subsets[i][1])
    knns.append(knn)

In [92]:
#create an array for the predictions of each type of classifier on the test set to store for voting later
predictions = []
rfpredictions = []
knnpredictions = []
for dt in dts:
    pred = dt.predict(features_test)
    predictions.append(pred)
for rf in rfs:
    pred = rf.predict(features_test)
    rfpredictions.append(pred)
for knn in knns:
    pred = knn.predict(features_test)
    knnpredictions.append(pred)

In [93]:
import statistics
#create an array for each type of classifier's bagging ensemble prediction
baggingpred = []
rfbaggingpred = []
knnbaggingpred = []
for i in range(len(features_test)):
    votes = [predictions[j][i] for j in range(len(predictions))]#inner loop that iterates from 1 to j(10) to gather each classifier's prediction for row i 
    rfvotes = [rfpredictions[j][i] for j in range(len(rfpredictions))]
    knnvotes = [knnpredictions[j][i] for j in range(len(knnpredictions))]
    mode = statistics.mode(votes)# gather highest voted prediction of each bagging ensemble
    rfmode = statistics.mode(rfvotes)
    knnmode = statistics.mode(knnvotes)
    baggingpred.append(mode)#add bagging ensemble prediction to array to measure accuracy against actual value
    rfbaggingpred.append(rfmode)
    knnbaggingpred.append(knnmode)

In [94]:
#print accuracy score of each bagging ensemble
print('Accuracy of decision tree model after bagging: %.3f' % accuracy_score(target_test, baggingpred))
print('Accuracy of random forest model after bagging: %.3f' % accuracy_score(target_test, rfbaggingpred))
print('Accuracy of KNN model after bagging: %.3f' % accuracy_score(target_test, knnbaggingpred))

Accuracy of decision tree model after bagging: 0.910
Accuracy of random forest model after bagging: 0.912
Accuracy of KNN model after bagging: 0.506


# **Step 4) Test using 10-fold cross validation**

In [95]:
#split whole dataset into 10 folds
folds = np.array_split(range(len(features_all_imputed)), 10)

accuracies = []
rfaccuracies = []
knnaccuracies = []
#For each fold, have it as a test set and the rest as training set
for i in range(10):
    test_idx = folds[i]
    train_idx = np.concatenate([folds[j] for j in range(10) if j != i])#loop iterates from 1 to 10 and for each fold it is set as test set and other 9 as training sets
    features_train, features_test,target_train, target_test = features_all_imputed[train_idx], features_all_imputed[test_idx],target[train_idx], target[test_idx]
    
    # Bagging (same code as Step 3)
    subsets = []
    target_train = np.array(target_train)
    for j in range(10):
        chosen = np.random.choice(len(features_train), 2000, replace=True)
        features_subset = features_train[chosen]
        target_subset = target_train[chosen]
        subsets.append((features_subset, target_subset))
    
    dts = []
    rfs=[]
    knns=[]
    for i in range(len(subsets)):
      dt = DecisionTreeClassifier(criterion='entropy')
      dt.fit(subsets[i][0], subsets[i][1])
      dts.append(dt)
    for i in range(len(subsets)):
      rf = RandomForestClassifier(criterion='entropy',n_estimators=75,max_depth=None)
      rf.fit(subsets[i][0], subsets[i][1])
      rfs.append(rf)
    for i in range(len(subsets)):
      knn = KNeighborsClassifier(n_neighbors=150)
      knn.fit(subsets[i][0], subsets[i][1])
      knns.append(knn)
    
    predictions = []
    rfpredictions = []
    knnpredictions = []
    for dt in dts:
      pred = dt.predict(features_test)
      predictions.append(pred)
    for rf in rfs:
      pred = rf.predict(features_test)
      rfpredictions.append(pred)
    for knn in knns:
      pred = knn.predict(features_test)
      knnpredictions.append(pred)
       
    baggingpred = []
    rfbaggingpred = []
    knnbaggingpred = []
    for i in range(len(features_test)):
      votes = [predictions[j][i] for j in range(len(predictions))]
      rfvotes = [rfpredictions[j][i] for j in range(len(rfpredictions))]
      knnvotes = [knnpredictions[j][i] for j in range(len(knnpredictions))]
      mode = statistics.mode(votes)
      rfmode = statistics.mode(rfvotes)
      knnmode = statistics.mode(knnvotes)
      baggingpred.append(mode)
      rfbaggingpred.append(rfmode)
      knnbaggingpred.append(knnmode)
    
    # Add accuracy score of each bagging ensemble to array to get mean
    accuracies.append(accuracy_score(target_test, baggingpred))
    rfaccuracies.append(accuracy_score(target_test, rfbaggingpred))
    knnaccuracies.append(accuracy_score(target_test, knnbaggingpred))

# Print the mean accuracy score
print('Accuracy score of bagged decision tree classifier tested using 10-fold cross validation: %.3f' % np.mean(accuracies))
print('Accuracy score of bagged random forest classifier tested using 10-fold cross validation: %.3f' % np.mean(rfaccuracies))
print('Accuracy score of bagged KNN classifier tested using 10-fold cross validation: %.3f' % np.mean(knnaccuracies))


Accuracy score of bagged decision tree classifier tested using 10-fold cross validation: 0.902
Accuracy score of bagged random forest classifier tested using 10-fold cross validation: 0.907
Accuracy score of bagged KNN classifier tested using 10-fold cross validation: 0.514


In [96]:
#split whole dataset into 10 folds
folds = np.array_split(range(len(features_all_imputed)), 10)

accuracies = []
rfaccuracies = []
knnaccuracies = []
#For each fold, have it as a test set and the rest as training set
for i in range(10):
    test_idx = folds[i]
    train_idx = np.concatenate([folds[j] for j in range(10) if j != i])#loop iterates from 1 to 10 and for each fold it is set as test set and other 9 as training sets
    features_train, features_test,target_train, target_test = features_all_imputed[train_idx], features_all_imputed[test_idx],target[train_idx], target[test_idx]
    #DT Classifier
    dt = DecisionTreeClassifier(criterion='entropy')
    dt.fit(features_train,target_train)
    dtpred = dt.predict(features_test)
    accuracies.append(accuracy_score(target_test, dtpred))
    #RF Classifier
    rf = RandomForestClassifier(criterion='entropy',n_estimators=75,max_depth=None)
    rf.fit(features_train,target_train)
    rfpred = rf.predict(features_test)
    rfaccuracies.append(accuracy_score(target_test, rfpred))
    #KNN Classifier
    knn=KNeighborsClassifier(n_neighbors=150)
    knn.fit(features_train,target_train)
    knnpred = knn.predict(features_test)
    knnaccuracies.append(accuracy_score(target_test, knnpred))

print('Accuracy score of decision tree classifier tested using 10-fold cross validation: %.3f' % np.mean(accuracies))
print('Accuracy score of random forest classifier tested using 10-fold cross validation: %.3f' % np.mean(rfaccuracies))
print('Accuracy score of KNN classifier tested using 10-fold cross validation: %.3f' % np.mean(knnaccuracies))

Accuracy score of decision tree classifier tested using 10-fold cross validation: 0.875
Accuracy score of random forest classifier tested using 10-fold cross validation: 0.911
Accuracy score of KNN classifier tested using 10-fold cross validation: 0.498
